AI Research Agent - Ollama
### https://huggingface.co/docs/smolagents

In [24]:
import pandas as pd

SmolAgents : CodeAgent & WebSearchTool

In [25]:
from smolagents import CodeAgent, WebSearchTool, LiteLLMModel

Select Model

In [26]:
model = LiteLLMModel(
    # model_name="smolagents/llama-2-7b-chat",
    model_id="ollama_chat/gemma3:latest",
    max_tokens=1024,
    temperature=0.1,
)

In [27]:
agent = CodeAgent(
    model=model,
    tools=[
        WebSearchTool(
            # name="web_search",
            # description="Search the web for information.",
            # max_results=5,
            # # search_engine="google",
        ),
    ],
    use_structured_outputs_internally=True
    )

Run Agent

In [28]:
questions = [
    "What is the capital of France?",
    "What is the population of Japan?",
    "What is the GDP of Germany?",
    "What is the tallest mountain in the world?",
    "What is the currency of Brazil?",
]

In [29]:
result = agent.run(questions[-2])

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the tallest mountain in the world?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/gemma3:latest ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  answer = search(query="tallest mountain in the world")                                                           
  print(answer)                                                                                                    
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'answer = search(query="tallest mountain in the world")' due to: InterpreterError: 
Forbidden function evaluation: 'search' is not among the explicitly allowed tools or defined/imported in the 
preceding code

[Step 1: Duration 9.41 seconds| Input tokens: 1,981 | Output tokens: 64]

...

[Step 2: Duration 3.33 seconds| Input tokens: 4,185 | Output tokens: 139]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  answer = "Mount Everest"                                                                                         
  print(answer)                                                                                                    
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Mount Everest

Out - Final answer: Mount Everest

[Step 3: Duration 3.14 seconds| Input tokens: 6,622 | Output tokens: 215]

In [30]:
result

'Mount Everest'

Building Dataset

In [31]:
subjectnkeys = ["fact","word_count"]

In [32]:
topics = [("Nepal", subjectnkeys), ("Japan", subjectnkeys), ("USA", subjectnkeys)]

In [33]:
rows = []
for topic, keys in topics:
    prompt = (
        f"Give one {keys[0]} about {topic}, and the {keys[1]} of that {keys[0]}."
    )
    output = agent.run(prompt)
    if isinstance(output, dict):
        output["subject"] = topic
        rows.append(output)
    else:
        print(f"Unexpected output for topic {topic}: {output}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give one fact about Nepal, and the word_count of that fact.                                                     │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/gemma3:latest ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  for country in ["Nepal"]:                                                                                        
      fact = wiki(query=f"fact about {country}")                                                                   
      print(fact)                                                                                                  
      word_count = len(fact.split())                                                                               
      print(f"Word count: {word_count}")                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'for country in ["Nepal"\]:
    fact = wiki(query=f"fact about {country}")
    print(fact)
    word_count = len(fact.split())
    print(f"Word count: {word_count}")' due to: InterpreterError: Forbidden function evaluation: 'wiki' is not 
among the explicitly allowed tools or defined/imported in the preceding code

[Step 1: Duration 4.84 seconds| Input tokens: 1,987 | Output tokens: 119]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  fact = "The United States of America is a country."                                                              
  word_count = len(fact.split())                                                                                   
  final_answer(f"{word_count}")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 8

[Step 4: Duration 3.15 seconds| Input tokens: 9,576 | Output tokens: 399]

Unexpected output for topic USA: 8


Add to dataframe

In [34]:
data = pd.DataFrame(rows)

In [35]:
data

,fact,word_count,subject
0,Nepal is a country in South Asia.,7,Nepal


### https://github.com/huggingface/smolagents
### 